In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Training Data**

In [ ]:
df_train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
df_train.head()

**Testing Data**

In [ ]:
df_test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
df_test.head()

**Train and Test Images**

In [ ]:
X_train = df_train.drop('label', axis=1)
y_train = df_train['label']

**Visualizing the Data**

In [ ]:
def viz_num(num):
    #Reshape the 768 values to a 28x28 image
    image = X_train.values[num].reshape([28,28])
    plt.title('Sample: %d  Label: %d' % (num, y_train[num]))
    plt.imshow(image, cmap=plt.get_cmap('gray'))
    plt.show()

In [ ]:
viz_num(1111)

**Formatting the Data**

In [ ]:
X_train = X_train/255.0
df_test = df_test/255.0
X_train = X_train.values.reshape(-1, 28, 28, 1)
df_test = df_test.values.reshape(-1, 28, 28, 1)

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import RMSprop

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

In [ ]:
y_train = keras.utils.to_categorical(y_train)
y_validation = keras.utils.to_categorical(y_validation)

**Modeling**

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(28 ,28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same',))
model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same',))
model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same',))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
early_stopping = EarlyStopping(min_delta=0.001, patience=5, restore_best_weights=True)

In [ ]:
history = model.fit(
    x=X_train, 
    y=y_train, 
    batch_size=32, 
    epochs=20, 
    callbacks=[early_stopping], 
    validation_data=(X_validation, y_validation))

In [ ]:
score = (max(history.history['val_accuracy']))
print(score)

**Submission**

In [ ]:
sample_submission = pd.read_csv('../input/digit-recognizer/sample_submission.csv')

In [ ]:
sample_submission

In [ ]:
y_test = model.predict(df_test)
y_test = np.argmax(y_test, axis=1)
y_test

In [ ]:
submission = pd.DataFrame(columns=['ImageId', 'Label'])

In [ ]:
submission['ImageId'] = range(1, 1+len(df_test))
submission['Label'] = y_test

In [ ]:
submission

In [ ]:
submission.to_csv('submissions0.csv', index=False)